# Importando as bibliotecas

In [9]:
#!pip install metapy-toolbox
#!pip install --upgrade EASYPLOT-TOOLBOX
#!pip install tqdm
from metapy_toolbox import genetic_algorithm_01, loss_function_mse
from my_example import my_function_diana_inverse_problem
from scipy import interpolate
import pandas as pd
from EASYPLOT_TOOLBOX import *

# Carregando o dataset experimental

In [5]:
caminho = './DadosExperimentaisMed_F15.out'
file = pd.read_csv(caminho, sep='\s+', header=0, names=['Desl', 'P'], dtype={'Desl': float, 'P': float})
x_exp = file['Desl'].tolist()
y_exp = file['P'].tolist()
none_variable = interpolate.interp1d(x_exp, y_exp)

# Configuração do método

In [3]:
setup = {   
            'number of iterations': 20,
            'number of population': 10,
            'number of dimensions': 4,
            'x pop lower limit': [0.50*3.93, 0.10*0.50*3.93, 0.0005, 0.006],
            'x pop upper limit': [2.00*3.93, 3.93, 0.005, 0.0583],
            'none variable': none_variable,
            'objective function': my_function_diana_inverse_problem,
            'algorithm parameters': {
                                     'selection': {'type': 'roulette'},
                                     'crossover': {'crossover rate (%)': 85, 'type':'linear'},
                                     'mutation': {'mutation rate (%)': 15, 'type': 'hill climbing', 'cov (%)': 15, 'pdf': 'gaussian'},
                                    }
        }

from metapy_toolbox import initial_population_01
init_pop = initial_population_01(setup['number of population'],
                                setup['number of dimensions'],
                                setup['x pop lower limit'],
                                setup['x pop upper limit'])

# Seed
seed = None

# Rodando o método de otimização

In [4]:
settings = [setup, init_pop, seed]
df_all_results, df_resume, time_cost, report = genetic_algorithm_01(settings)

# Relatórios

In [6]:
df_resume.tail(5)

,X_0,X_1,X_2,X_3,OF BEST,FIT BET,ID BEST,X_0,X_1,X_2,X_3,OF WORST,FIT WORST,ID WORST,OF AVG,FIT AVG,ITERATION,neof
16,7.86,0.60518,0.003009,0.6,0.000476,0.999524,5,7.86,0.256153,0.003518,0.984358,0.051419,0.951096,3,0.021107,0.979713,16,37
17,7.86,0.60518,0.003009,0.6,0.000476,0.999524,5,7.86,0.256153,0.003518,0.984358,0.051419,0.951096,3,0.021107,0.979713,17,37
18,7.86,0.60518,0.003009,0.6,0.000476,0.999524,5,7.86,0.256153,0.003518,0.984358,0.051419,0.951096,3,0.021107,0.979713,18,38
19,7.86,0.60518,0.003009,0.6,0.000476,0.999524,5,7.86,0.256153,0.003518,0.984358,0.051419,0.951096,3,0.021107,0.979713,19,38
20,7.86,0.60518,0.003009,0.6,0.000476,0.999524,5,7.86,0.256153,0.003518,0.984358,0.051419,0.951096,3,0.019413,0.981349,20,41


In [ ]:
x_best = list(df_resume.iloc[-1])[:setup['number of dimensions']]
x_best

# Salvando relatóiro

### txt

In [7]:
filename = "report_genetic_algorithm_01.txt"
with open(filename, "w") as f:
    f.write(report)

### xlsx

In [8]:
filename = "report_genetic_algorithm_01.xlsx"
df_resume.to_excel(filename, index=False)

# Análise gráfica da otm

In [ ]:
# Data
DF =  {'x0': df_resume['neof'],
        'y0': df_resume['OF BEST']
}

# Chart setup
CHART_CONFIG = {
              'NAME': 'of_result_analise_inversa_2d',
              'WIDTH': 20, 
              'HEIGHT': 10,
              'MARKER': ['s'],
              'MARKER SIZE': 3,
              'LINE WIDTH': 4,
              'LINE STYLE': ['--'],
              'X AXIS LABEL': 'neof',
              'X AXIS SIZE': 20,
              'Y AXIS LABEL': 'dead load ($kN/m$)',
              'Y AXIS SIZE': 20,
              'AXISES COLOR': '#000000',
              'LABELS SIZE': 20,
              'LABELS COLOR': '#000000',
              'CHART COLOR': ['#000000'],
              'ON GRID?': True,
              'LEGEND': [None], # or without legend 'LEGEND': [None]
              'LOC LEGEND': 'upper right',
              'SIZE LEGEND': 12,
              'Y LOG': False,
              'X LOG': False,
              'DPI': 200, 
              'EXTENSION': 'png'
             }

# Data statement 
DATA = {'DATASET': DF}

# Call function
line_chart(DATASET = DATA, PLOT_SETUP = CHART_CONFIG)


# Comparação gráfica

In [13]:
mse, x_num, y_num = my_function_diana_inverse_problem_2(x_best, none_variable)

# Data
DF =  {
        'x0': x_num,
        'y0': y_num,
        'x1': x_exp,
        'y1': y_exp,
      }

# Chart setup
CHART_CONFIG = {
              'NAME': 'resultado-comparacao',
              'WIDTH': 20, 
              'HEIGHT': 10,
              'MARKER': [None, None],
              'MARKER SIZE': 3,
              'LINE WIDTH': 3,
              'LINE STYLE': ['-', '-'],
              'X AXIS LABEL': 'displacement ($mm$)',
              'X AXIS SIZE': 20,
              'Y AXIS LABEL': 'Load ($kN$)',
              'Y AXIS SIZE': 20,
              'AXISES COLOR': '#000000',
              'LABELS SIZE': 20,
              'LABELS COLOR': '#000000',
              'CHART COLOR': ['red', 'blue'],
              'ON GRID?': True,
              'LEGEND': ['numerical model', 'true data'], # or without legend 'LEGEND': [None]
              'LOC LEGEND': 'upper right',
              'SIZE LEGEND': 12,
              'Y LOG': False,
              'X LOG': False,
              'DPI': 200, 
              'EXTENSION': 'png'
             }

# Data statement 
DATA = {'DATASET': DF}

# Call function
line_chart(DATASET = DATA, PLOT_SETUP = CHART_CONFIG)


7.25